# DOWNLOAD APK WITH SEARCH STRING

In [48]:
import os
import time
import glob
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import requests
import pandas as pd
import os
import urllib.parse
import threading
from time import sleep
from threading import Thread
import glob
import shutil

In [49]:
def checkFileOrDirectoryExist(path):
    if os.path.exists(path):
        return True
    else:
        return False

In [50]:
def createDirectory(PARENT_PATH: str,search_string: str):
    directory_path = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"
    check_directory_exist = checkFileOrDirectoryExist(directory_path)
    if(check_directory_exist == False):
        os.mkdir(directory_path)
        return "Mkdir directory "+directory_path +" success!"
    else:
        return "Directory "+directory_path +" is existed!"

In [51]:
def createEmptyCSVFile(PARENT_PATH: str,search_string: str):
    directory_path = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search\\"
    file_name = str(search_string.replace(' ', '-'))+"_search.csv"
    check_file_exist = checkFileOrDirectoryExist(directory_path+file_name)
    if(check_file_exist == False):
        df = pd.DataFrame(columns=['number','pkg_name','apkfile_name','download','apk_link','manifest_permission'])
        df.to_csv(directory_path+file_name,index=False)
        return "Create file "+directory_path+file_name +" success!"
    else:
        return "File "+directory_path+file_name +" is existed!"

In [52]:
def removeInvalidCharacter(filename):
    invalid = ' <>:"/\|?*!+;,=()'
    for char in invalid:
        filename = filename.replace(char,'-')
    return filename

In [53]:
def waitingDownload(PARENT_PATH: str,search_string: str):
    path = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp\\*.apk"
    path_xapk = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp\\*.xapk"
    files = glob.glob(path)
    files_xapk = glob.glob(path_xapk)
    #print(files)
    if(len(files)!=0):
        return True
    elif(len(files_xapk)!=0):
        return True
    else:
        return False

In [54]:
def fileAPKPathFunc(PARENT_PATH: str,search_string: str):
    path = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp\\*.apk"
    path_xapk = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp\\*.xapk"
    files = glob.glob(path)
    files_xapk = glob.glob(path_xapk)
    #print(files)
    if((len(files)!=0)):
        return files[0],"apk"
    elif((len(files_xapk)!=0)):
        return files_xapk[0],"xapk"
    else:
        return "",""

In [55]:
def deleteFile(dir):
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

In [56]:
# 0. Prepare environment
# 0.1 Constant
APK_WEB_LINK = "https://apkcombo.com/search/"
PARENT_PATH = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\"
# 0.2 Input search
search_string = "tiny png image compressor"
url = APK_WEB_LINK+str(search_string.replace(' ', '-'))
print("URL: "+url)
# 0.3 Create directory
create_directory  = createDirectory(PARENT_PATH,search_string)
print(create_directory)
# 0.4 Create CSV
createCSVFileStatus = createEmptyCSVFile(PARENT_PATH,search_string)
print(createCSVFileStatus)
# 0.4 Read CSV file
csv_file_name = str(search_string.replace(' ', '-'))+"_search.csv"
csv_path = PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\"+csv_file_name
df = pd.read_csv(csv_path,low_memory=False)

URL: https://apkcombo.com/search/tiny-png-image-compressor
Mkdir directory C:\Users\ASUS\anaconda3\phd_implement\tiny-png-image-compressor_search success!
Create file C:\Users\ASUS\anaconda3\phd_implement\tiny-png-image-compressor_search\tiny-png-image-compressor_search.csv success!


In [57]:
try:
    # 1.Get list app download link
    # 1.1 Setup selenium
    chromedriver = 'chromedriver.exe'
    os.environ["webdriver.chrome.driver"] = chromedriver
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument('--ignore-ssl-errors=yes')
    chrome_options.add_argument('--ignore-certificate-errors')
    chrome_options.add_argument('--no-proxy-server')
    chrome_options.add_argument('--headless=new')
    prefs = {
        'profile.default_content_setting_values.automatic_downloads': 1,
        "download.default_directory" : PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp"
    }
    chrome_options.add_experimental_option("prefs", prefs)
    # 1.2 Open Web Page
    web = webdriver.Chrome(options=chrome_options)
    web.maximize_window()
    web.get(url)
    time.sleep(7)
    # 1.3 Agree privacy
    try:
        privacy_agree=web.find_element(By.CSS_SELECTOR,"button[class='sc-ifAKCX ljEJIv']")
        privacy_agree.click()
        time.sleep(5)
    except NoSuchElementException:
        print("Element does not exist")
    # 1.4 Get google list apk 
    google_lists=web.find_elements(By.CSS_SELECTOR,"a[class='l_item']")
    download_link_array = []
    for i in range(len(google_lists)):
        pkg_name=web.find_elements(By.CSS_SELECTOR,"a[class='l_item']")[i].get_attribute('href')
        download_link_array.append(pkg_name+"download/apk")
    print(download_link_array)
    print("We have total: " + str(len(download_link_array))+" apks!")
    web.close()
    # 2. Download apk
    for j in range(len(download_link_array)):
        print("----------------------------------[APK-"+str(j)+"]----------------------------------")
        # 2.1 Open web page
        web = webdriver.Chrome(options=chrome_options)
        web.maximize_window()
        web.get(download_link_array[j])
        time.sleep(7)
        # 2.2 Agree privacy
        try:
            privacy_agree=web.find_element(By.CSS_SELECTOR,"button[class='sc-ifAKCX ljEJIv']")
            privacy_agree.click()
            time.sleep(5)
            df.loc[j,"apk_link"]=str(download_link_array[j])
        except NoSuchElementException:
            print("Element does not exist")
        # 2.3 Find download link 
        try:
            apk_link=web.find_element(By.CLASS_NAME, "variant").get_attribute('href')
            apkfile_name = web.find_element(By.CLASS_NAME, "vername").text
            apkfile_type = web.find_element(By.CLASS_NAME, "type-apk").text
            print("[Link-"+str(j)+"] "+apk_link)
            apkfile_name=removeInvalidCharacter(apkfile_name)
            print("File name: "+apkfile_name)
            pkg_name_arr = (download_link_array[j]).split("/")
            pkg_name = str(pkg_name_arr[4])
            print("Package ID: "+pkg_name)
            df.loc[int(j),"pkg_name"]=pkg_name
            if(apkfile_type!=""):
                df.loc[int(j),"apkfile_name"]=str(apkfile_name+"."+(apkfile_type).lower())
            else:
                df.loc[int(j),"apkfile_name"]=str(apkfile_name+".xapk")
        except NoSuchElementException:
            print("[Link-"+str(j)+"] App does not exist")
            continue
        # 2.2 Download APK  files from download_link
        def downloadAPK():
            web.get(apk_link)
        thread= Thread(target=downloadAPK())
        thread.start()
        print('Waiting for the apk downloading...')
        print('Start timeout...')
        timeout = time.time() + 60*15 # timeout after 15 minutes
        while True:
            if((waitingDownload(PARENT_PATH,search_string)) or (time.time() > timeout)):
                break
        thread.join()
        print('Download completed')
        time.sleep(7)
        df.loc[int(j),"download"]="Done"
        df.loc[int(j),"number"]=int(j)
        time.sleep(3)
        fileAPKPath,fileExtension=fileAPKPathFunc(PARENT_PATH,search_string)
        if((fileAPKPath!="") and (fileExtension!="")):
            if(not checkFileOrDirectoryExist(PARENT_PATH+str(search_string.replace(' ', '-'))+"_search\\"+str(apkfile_name)+"."+str(fileExtension))):
                time.sleep(15)
                os.rename(fileAPKPath, PARENT_PATH+str(search_string.replace(' ', '-'))+"_search\\"+str(apkfile_name)+"."+str(fileExtension))
                time.sleep(15)
                deleteFile(PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp")
                time.sleep(5)
            else:
                deleteFile(PARENT_PATH+str(search_string.replace(' ', '-'))+"_search"+"\\tmp")
                time.sleep(10)
        web.close()
        time.sleep(3)
except Exception as e: 
    df.to_csv(csv_path,index=False)
    export = pd.read_csv(csv_path,low_memory=False)
    print(export)
    print(e)

Element does not exist
['https://apkcombo.com/tiny-png-image-compressor/com.bloomers.tinypng/download/apk', 'https://apkcombo.com/tiny-png-image-compressor-pro/com.bloomers.tinypng.pro/download/apk', 'https://apkcombo.com/tiny-png-image-compressor/com.bloomers.tinypng/download/apk', 'https://apkcombo.com/image-compressor-png-jpg-gif/com.photo.resize_crop_compress_convert_image/download/apk', 'https://apkcombo.com/image-compressor-resize-image/com.imageresizer.imagecompressor/download/apk', 'https://apkcombo.com/photo-compressor-and-resizer/com.photocompress.photoeditor/download/apk', 'https://apkcombo.com/photo-compress-image-resize/aculix.bulk.image.compressor/download/apk', 'https://apkcombo.com/tiny-image-image-compressor/appbuck3t.tinypng.imagecompressor.photominifier/download/apk', 'https://apkcombo.com/photo-resize-compress-crop/com.jsk.photoresizer/download/apk', 'https://apkcombo.com/photo-compressor-compress-png/com.as.compressor.imagecompress.mbtokb.freeapp/download/apk', 'ht

Waiting for the apk downloading...
Start timeout...
Download completed
----------------------------------[APK-9]----------------------------------
Element does not exist
[Link-9] https://apkcombo.com/d?u=aHR0cHM6Ly9kLTEwLndpbnVkZi5jb20vYi9BUEsvWTI5dExtRnpMbU52YlhCeVpYTnpiM0l1YVcxaFoyVmpiMjF3Y21WemN5NXRZblJ2YTJJdVpuSmxaV0Z3Y0Y4ME1EWmZaV0kyTnpNNE9BP2s9MmQ5Njg4N2Y0M2E3OWY2NTY1YWZmZDhhMjFhYmM5NzQ2NGU0N2JkZSZfZm49VUdodmRHOHJRMjl0Y0hKbGMzTnZjaVV6UVN0RGIyMXdjbVZ6Y3l0UVRrZGZOQzR3TGpaZllYQnJZMjl0WW04dVkyOXRMbUZ3YXclM0QlM0Q=&fp=d0a90adbc4490b3c772a7f166fcc8a8a&ip=171.233.236.250
File name: Photo-Compressor--Compress-PNG-4.0.6
Package ID: com.as.compressor.imagecompress.mbtokb.freeapp
Waiting for the apk downloading...
Start timeout...
Download completed
----------------------------------[APK-10]----------------------------------
Element does not exist
[Link-10] https://download.apkcombo.com/com.csdeveloper.imagecompressor/Image%20Compressor,%20Size%20in%20KB_5.6_apkcombo.com.apk?ecp=Y29tLmNzZGV2

Waiting for the apk downloading...
Start timeout...
Download completed
----------------------------------[APK-22]----------------------------------
Element does not exist
[Link-22] https://download.apkcombo.com/com.adobe.scan.android/Adobe%20Scan_23.08.14-regular_apkcombo.com.apk?ecp=Y29tLmFkb2JlLnNjYW4uYW5kcm9pZC8yMy4wOC4xNC1yZWd1bGFyLzIzMjYzNzY3MC5kYTUyYmNhN2UzOTgyOTFlOWY2MTQ4MDc2YjRlN2ZiZjI2YWQwZWFjLmFwaw==&iat=1692523650&sig=ca0c6e2f2bd7ea7a408223196727cff6&size=83079162&from=cf&version=latest&lang=en&fp=d0a90adbc4490b3c772a7f166fcc8a8a&ip=171.233.236.250
File name: Adobe-Scan-23.08.14-regular
Package ID: com.adobe.scan.android
Waiting for the apk downloading...
Start timeout...
Download completed
----------------------------------[APK-23]----------------------------------
Element does not exist
[Link-23] https://download.apkcombo.com/com.ashaquavision.imagecompressor.imageconverter.reducephotosize/Image%20Converter:%20JPG%2FJPEG,%20PNG_23.MAY.28.0.22_apkcombo.com.apk?ecp=Y29tLmFza

In [58]:
# 3. Export CSV
df.to_csv(csv_path,index=False)
export = pd.read_csv(csv_path,low_memory=False)
export

,number,pkg_name,apkfile_name,download,apk_link,manifest_permission
0,0,com.bloomers.tinypng,Tiny-Png-Image-Compressor-3.0.apk,Done,NaN,NaN
1,2,com.bloomers.tinypng,Tiny-Png-Image-Compressor-3.0.apk,Done,NaN,NaN
2,3,com.photo.resize_crop_compress_convert_image,Image-Compressor-png-jpg-gif-2.3.1.apk,Done,NaN,NaN
3,4,com.imageresizer.imagecompressor,Image-Compressor--Resize-Image-4.8.apk,Done,NaN,NaN
4,5,com.photocompress.photoeditor,Photo-Compressor-and-Resizer-1.0.9.apk,Done,NaN,NaN
5,6,aculix.bulk.image.compressor,Photo-Compress---Image-Resize-3.11.apk,Done,NaN,NaN
6,7,appbuck3t.tinypng.imagecompressor.photominifier,Tiny-Image---Image-Compressor-1.0.1.apk,Done,NaN,NaN
7,8,com.jsk.photoresizer,Photo-Resize---Compress--Crop-1.1.1.apk,Done,NaN,NaN
8,9,com.as.compressor.imagecompress.mbtokb.freeapp,Photo-Compressor--Compress-PNG-4.0.6.apk,Done,NaN,NaN
9,10,com.csdeveloper.imagecompressor,Image-Compressor--Size-in-KB-5.6.apk,Done,NaN,NaN
